In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import build_resnet

In [ ]:
def format_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return  image, label

In [ ]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes
print(f"number of training examples:{num_examples}, number of classes: {num_classes}")

In [ ]:
BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = raw_test.map(format_image).batch(1)

for image_batch, label_batch in train_batches.take(1):
    pass

print(f"image shape: {image_batch.shape})

In [ ]:
resnet = build_resnet.build_resnet()

In [ ]:
resnet.compile(optimizer="adam",
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
hist = resnet.fit(train_batches,
                 epochs=10,
                 validation_data=validation_batches)